In [3]:
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score
import pandas as pd
import numpy as np


In [4]:
#import data here 

train_data = pd.read_csv('./train_noclean_prices_dropped.csv')
test_data = pd.read_csv('./test_noclean_prices_dropped.csv')

y_train = train_data.loc[:,'price']
train_data = train_data.drop(columns='price')
num_features = len(train_data.columns)
features = list(train_data.columns[2:num_features])
X_train = train_data.loc[:,features]

features = list(test_data.columns[2:num_features])

X_test = test_data.loc[:,features]
test_labels = test_data.loc[:,'id']

test_labels = test_labels.to_frame()

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

kf = KFold(n_splits = 10, shuffle=True)


X = np.concatenate([X_train], axis= 0)
y = np.concatenate([y_train], axis= 0)

for train, test in kf.split(X_train):
    X_train_kf, X_test_kf = X[train],X[test]
    y_train_kf, y_test_kf = y[train],y[test]
    
    params = {'n_estimators': 1000, 'min_samples_split': 2, 'min_samples_leaf': 1, 'max_features': 'auto', 'max_depth': 20, 'criterion': 'gini', 'bootstrap': True}

    clf = RandomForestClassifier()
    clf.set_params(**params)
    clf.fit(X_train_kf,y_train_kf)
    accuracy = clf.score(X_test_kf, y_test_kf)
    print(accuracy)

In [5]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

kf = KFold(n_splits = 10, shuffle=True)

param_grid = {'bootstrap': [True, False],
 'criterion': ['gini', 'entropy'],
 'max_depth': [2, 5, 10, 20, 30, None],
 'max_features': ['auto', 'log2', 3, 8, None],
 'min_samples_leaf': [1, 2, 4],
 'min_samples_split': [2, 5, 10],
 'n_estimators': [50, 100, 200, 400, 600, 800, 1000, 1500]}
X = np.concatenate([X_train], axis= 0)
y = np.concatenate([y_train], axis= 0)


clf = RandomForestClassifier()  
rsearch = RandomizedSearchCV(estimator=clf, param_distributions=param_grid, cv=kf, n_iter = 1000, n_jobs = -1, verbose=10)
clf_model = rsearch.fit(X=X, y=y)

print(clf_model.best_params_, clf_model.best_score_)


Fitting 10 folds for each of 1000 candidates, totalling 10000 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:    6.8s
[Parallel(n_jobs=-1)]: Done  18 tasks      | elapsed:    7.7s
[Parallel(n_jobs=-1)]: Done  29 tasks      | elapsed:   12.2s
[Parallel(n_jobs=-1)]: Done  40 tasks      | elapsed:   27.3s
[Parallel(n_jobs=-1)]: Done  53 tasks      | elapsed:   34.8s
[Parallel(n_jobs=-1)]: Done  66 tasks      | elapsed:   49.6s
[Parallel(n_jobs=-1)]: Done  81 tasks      | elapsed:  1.3min
[Parallel(n_jobs=-1)]: Done  96 tasks      | elapsed:  1.9min
[Parallel(n_jobs=-1)]: Done 113 tasks      | elapsed:  2.2min
[Parallel(n_jobs=-1)]: Done 130 tasks      | elapsed:  2.3min
[Parallel(n_jobs=-1)]: Done 149 tasks      | elapsed:  2.6min
[Parallel(n_jobs=-1)]: Done 168 tasks      | elapsed:  2.9min
/opt/conda/lib/python3.7/site-packages/joblib/externals/loky/process_executor.py:691: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused 

{'n_estimators': 100, 'min_samples_split': 2, 'min_samples_leaf': 2, 'max_features': 'auto', 'max_depth': 20, 'criterion': 'entropy', 'bootstrap': False} 0.5578992144922345


In [29]:
print(clf_model.best_params_, clf_model.best_score_)
#params with prices added

{'n_estimators': 1500, 'min_samples_split': 5, 'min_samples_leaf': 1, 'max_features': 'auto', 'max_depth': 20, 'criterion': 'gini', 'bootstrap': False} 0.5590341921892724


In [16]:
print(clf_model.best_params_, clf_model.best_score_)
#params without changing max features

{'n_estimators': 400, 'min_samples_split': 5, 'min_samples_leaf': 1, 'max_features': 'auto', 'max_depth': None, 'criterion': 'gini', 'bootstrap': False} 0.5590306740355995


In [6]:
params = {'n_estimators': 100, 'min_samples_split': 2, 'min_samples_leaf': 2, 'max_features': 'auto', 'max_depth': 20, 'criterion': 'entropy', 'bootstrap': False}
clf = RandomForestClassifier()
clf.set_params(**params)
clf.fit(X,y)


RandomForestClassifier(bootstrap=False, criterion='entropy', max_depth=20,
                       min_samples_leaf=2)

In [7]:
y_pred1=clf.predict(X)
print(y_pred1)
print(accuracy_score(y, y_pred1))

[2 1 3 ... 3 3 1]
0.992872637124264


In [8]:
y_pred=clf.predict(X_test)

df = pd.DataFrame
test_labels['price'] = y_pred

test_labels['price'] = test_labels['price'].map(lambda x: int(x))

In [9]:
test_labels

,id,price
0,7715,2
1,13196,2
2,13194,2
3,4673,2
4,11325,1
...,...,...
4144,12921,4
4145,7174,1
4146,9240,3
4147,11663,1


In [10]:
test_labels.to_csv(r'./submission16.csv', index=False)